# Подключаем необходимые библиотеки

In [10]:
import sqlite3 #Подключаем sqlite
import xlrd, xlwt #Подключаем утилиты для импорта данных excel

# Инициализируем подключение и создаем базу

In [ ]:
#При запуске указать новую bd_name и выполнить последовательно все окна ниже
bd_name="Test3.db" 

In [11]:
conn=sqlite3.Connection(bd_name)

In [12]:
cur=conn.cursor()

# Создаем необходимые input-таблицы

In [13]:
cur.execute("""
     create table organisations_test1
     (
       row_id string,
x_mts_full_accnt_name string,
ou_type_cd string,
x_holding_number string,
x_mts_national_manager_id string,
last_upd_utc date,
par_ou_id string
     )
     """)

In [14]:

cur.execute("""
     create table contracts_test1
     (
       row_id string,
cust_stat_cd string,
ou_num string,
x_mts_full_accnt_name string,
last_upd_utc date,
par_ou_id string
     )
     """)

In [15]:


cur.execute("""
     create table users_test1
     (
       row_id string,
fst_name string,
last_name string,
mid_name string,
job_title string,
last_upd_utc date
     )
     """)

# Загружаем данные из книги Источники в созданные таблицы

In [16]:


rb = xlrd.open_workbook('Источники.xlsx')

sheet = rb.sheet_by_index(0)
sheet.row_values(0)[6]

'GJSVX'

In [17]:
for i in range(0,3):
    sheet=rb.sheet_by_index(i)
    print(sheet.name)

organizations
contracts
users


In [18]:
sheet=rb.sheet_by_index(0)

#Импортируем organizations
for i in range(0,9):
    query="""
    insert into organisations_test1 values
    ('{row_id}','{x_mts_full_accnt_name}', '{ou_type_cd}','{x_holding_number}','{x_mts_national_manager_id}',
     '{last_upd_utc}','{par_ou_id}'
    )
    """.format(
               row_id=sheet.row_values(i)[0],
               x_mts_full_accnt_name=sheet.row_values(i)[1],
               ou_type_cd=sheet.row_values(i)[2],
               x_holding_number=sheet.row_values(i)[3],
               x_mts_national_manager_id=sheet.row_values(i)[4],
               last_upd_utc=sheet.row_values(i)[5],
               par_ou_id=sheet.row_values(i)[6]
    )
    cur.execute(query)
    

In [19]:
#Импортируем contracts

sheet = rb.sheet_by_index(1)
for i in range(0,8):
    query="""
    insert into contracts_test1 values
    ('{row_id}','{cust_stat_cd}', '{ou_num}','{x_mts_full_accnt_name}','{last_upd_utc}','{par_ou_id}')
    """.format(
               row_id=sheet.row_values(i)[0],
               cust_stat_cd=sheet.row_values(i)[1],
               ou_num=sheet.row_values(i)[2],
               x_mts_full_accnt_name=sheet.row_values(i)[3],
               last_upd_utc=sheet.row_values(i)[4],
               par_ou_id=sheet.row_values(i)[5]
    )
    cur.execute(query)

In [20]:
#Импортируем users


sheet = rb.sheet_by_index(2)
for i in range(0,9):
    query="""
    insert into users_test1 values
    ('{row_id}','{fst_name}', '{last_name}','{mid_name}','{job_title}','{last_upd_utc}')
    """.format(
               row_id=sheet.row_values(i)[0],
               fst_name=sheet.row_values(i)[1],
               last_name=sheet.row_values(i)[2],
               mid_name=sheet.row_values(i)[3],
               job_title=sheet.row_values(i)[4],
               last_upd_utc=sheet.row_values(i)[5]
    )
    cur.execute(query)

# Формируем результирующую выборку согласно заданию

In [31]:
sqlite3.version

'2.6.0'

In [23]:
#sqlite3.version до 3 не поддерживает оконных функций. поэтому считаем связку row_id+last_upd_utc в каждой таблице первичным ключом
#и реализуем логику, используя особенность group by в sqlite3, позволяющую выводить в результат поля, не участвующие в группировке
cur.execute("""
  select
    cntr_number,
    holding_id,
    holding_name,
    x_holding_number,
    last_name||" "||fst_name||" "||mid_name as fio,
    job_title
  from
  (
  select 
    cntr.ou_num as cntr_number, 
    holding.row_id as holding_id,
    holding.x_mts_full_accnt_name as holding_name,
    holding.x_holding_number,
    users.fst_name,
    users.last_name,
    users.mid_name,
    users.job_title,
    max(cntr.last_upd_utc),
    max(urik.last_upd_utc),
    max(holding.last_upd_utc),
    max(nat_cntr.last_upd_utc),
    max(users.last_upd_utc)
  from contracts_test1 cntr
  join organisations_test1 urik
   on cntr.par_ou_id=urik.row_id
  join organisations_test1 nat_cntr
   on urik.par_ou_id=nat_cntr.row_id
  join organisations_test1 holding
    on nat_cntr.par_ou_id=holding.row_id
  join users_test1 users
    on holding.x_mts_national_manager_id=users.row_id
  where cntr.cust_stat_cd='Открыт'
  group by cntr.row_id, urik.row_id, nat_cntr.row_id, holding.row_id, users.row_id
  )
""")
cur.fetchall()

[(1784324873,
  'LJLKS',
  'Ромашка Холдинг',
  12345,
  'Лебедев Павел Александрович',
  'эксперт'),
 (1765467564,
  'YPGFD',
  'Василёк Inc.',
  12346,
  'Смирнов Николай Сергеевич',
  'специалист'),
 (1773845363,
  'LJLKS',
  'Ромашка Холдинг',
  12345,
  'Лебедев Павел Александрович',
  'эксперт')]

# Альтернативный скрипт с фильтрацией истории оконными функциями

# Закрываем соединение с базой

In [ ]:
conn.close()